In [ ]:
import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import curve_fit

COSMICSDB = "/exp/icarus/data/users/mvicenzi/timing-database/pmt_cosmics_timing_data/"
PATH = "/exp/icarus/data/users/mvicenzi/pmt-calibration/residualsdb/"

In [ ]:
def plot_comparison(f1, f2, label1, label2, b, r):

    df1 = pd.read_csv(f1)
    df2 = pd.read_csv(f2)

    fig, ax = plt.subplots(1,2, figsize=(16, 5))
    l1 = "{}\nMean: {:.2f} ns\nStd: {:.2f} ns".format(label1, np.mean(df1.mean_residual_ns),np.std(df1.mean_residual_ns))
    l2 = "{}\nMean: {:.2f} ns\nStd: {:.2f} ns".format(label2, np.mean(df2.mean_residual_ns),np.std(df2.mean_residual_ns))

    ax[0].hist(df1.mean_residual_ns, bins=b, linewidth=2, range=r, histtype="step", label=l1)
    ax[0].hist(df2.mean_residual_ns, bins=b, linewidth=2, range=r, histtype="step", label=l2)

    ax[0].set_xlabel("Time residuals [ns]")
    ax[0].set_ylabel("Number of PMTs")
    ax[0].grid(linestyle="dashed",alpha=0.5)

    channels = np.unique(df2.channel.to_numpy())
    diff = []
    for c in channels:
        new=(df1[df1.channel_id==c].mean_residual_ns.values[0])
        old=(df2[df2.channel==c].mean_residual_ns.values[0])
        diff.append(new-old)
    
    print("There are {} channels in both files".format(len(diff)))
    ax[1].hist(diff, bins=b, linewidth=2, range=(-4,4), histtype="step")
    
    ax[1].set_xlabel("New - old time residual [ns]")
    ax[1].set_ylabel("# of PMTs")
    ax[1].grid(linestyle="dashed",alpha=0.5)

    ax[0].legend()
    plt.show()

In [ ]:
def check_bias(f1, f2, label1, label2):

    df = pd.read_csv(f1)
    df2 = pd.read_csv(f2)
   
    # Verify potential presence of biases 
    fig, ax = plt.subplots(1,2, figsize=(16, 5))
    ax[0].errorbar( x=df.y, y=df.mean_residual_ns, yerr=df.emean_ns, marker='.', elinewidth=2.0, lw=0, label=label1)
    ax[0].errorbar( x=df2.y, y=df2.mean_residual_ns, yerr=df2.emean_ns, marker='.', elinewidth=2.0, lw=0, label=label2 )
    ax[1].errorbar( x=df.z, y=df.mean_residual_ns, yerr=df.emean_ns, marker='.', elinewidth=2.0, lw=0 , label=label1)
    ax[1].errorbar( x=df2.z, y=df2.mean_residual_ns, yerr=df2.emean_ns, marker='.', elinewidth=2.0, lw=0, label=label2)
    ax[0].set_ylabel("Cosmics Residuals [ns]", fontsize=16)
    ax[1].set_ylabel("Cosmics Residuals [ns]", fontsize=16)
    ax[0].set_xlabel("Y position [cm]", fontsize=16)
    ax[1].set_xlabel("Z position [cm]", fontsize=16)

    plt.tight_layout()
    ax[0].grid(alpha=0.5,linestyle="dashed")    
    ax[1].grid(alpha=0.5,linestyle="dashed")
    plt.legend()
    plt.show()

In [ ]:
# The fit function
def gaus(x,a,mean,sigma):
    return a*np.exp(-(x-mean)**2/(2*sigma**2))

# The fit strategy
def fitGaussian(hdf, target='mean_residual_ns', BINSIZE=0.5,RMIN=-10.0,RMAX=10., p0=(0.1, 1.)):

    nbins=int((RMAX-RMIN)/BINSIZE)
    ys,edges = np.histogram(hdf[target],bins=nbins,range=(RMIN,RMAX))
    xs=np.array([edges[i]+0.5*(edges[i+1]-edges[i]) for i in range(len(ys))])

    param = [np.max(ys), p0[0], p0[1] ]
    bounds = np.array([(param[0]*0.5,param[0]*1.5),
              (param[1]*0.2,param[1]*1.3),
              (param[2]*0.1,param[2]*1.4)])
    
    param,pcov = curve_fit(gaus, xs, ys, p0=param, bounds=(bounds[:,0],bounds[:,1]) )
    param_errors = np.diag(pcov)**0.5
    
    return xs, ys, param, param_errors

In [ ]:
def plot_validation(PATH, period, run, corr):

    nolaser = PATH + "{}/run{}_residuals_nocorr.csv".format(period, run)
    laser =  PATH + "{}/run{}_residuals_laseronly.csv".format(period, run)
    cosmics = PATH + "{}/run{}_residuals_lasercosmics.csv".format(period, run)

    dfnolaser = pd.read_csv(nolaser).set_index(["channel_id"])
    dflaser = pd.read_csv(laser).set_index(["channel_id"])
    dfcosmics = pd.read_csv(cosmics).set_index(["channel_id"])

    # Plotting residuals distribution with/without corrections
    fig = plt.figure(figsize=(8,5),dpi=200)

    rmin=-7
    rmax=7
    r=(rmin,rmax)
    s=0.3
    b=int((rmax-rmin)/s)

    xs1, ys1, param1, param_errors1 = fitGaussian(dfnolaser,BINSIZE=s,RMIN=rmin,RMAX=rmax)
    xs2, ys2, param2, param_errors2 = fitGaussian(dflaser,BINSIZE=s,RMIN=rmin,RMAX=rmax)
    xs3, ys3, param3, param_errors3 = fitGaussian(dfcosmics,BINSIZE=s,RMIN=rmin,RMAX=rmax)
    xss = np.arange(rmin,rmax,0.1)

    ### PLOT FITS
    l_nolaser = "Base sample"
    l_nolaserf= "$\mu$: {:.2f} ns\n$\sigma$: {:.2f} ns".format(param1[1],param1[2])
    l_laser = "Laser only"
    l_laserf = "$\mu$: {:.2f} ns\n$\sigma$: {:.2f} ns".format(param2[1],param2[2])
    l_cosmics = "Laser + cosmics"
    l_cosmicsf = "$\mu$: {:.2f} ns\n$\sigma$: {:.2f} ns".format(param3[1],param3[2])
    plt.hist(dfnolaser.mean_residual_ns, bins=b, linewidth=2, range=r, histtype="step", label=l_nolaser)
    plt.plot(xss,gaus(xss,*param1),"-.",color="darkblue",lw=3, label=l_nolaserf)
    plt.hist(dflaser.mean_residual_ns, bins=b, linewidth=2, range=r, histtype="step", label=l_laser)
    plt.plot(xss,gaus(xss,*param2),"-.",color="red",lw=3, label=l_laserf)
    plt.hist(dfcosmics.mean_residual_ns, bins=b, linewidth=2, range=r, histtype="step", label=l_cosmics)
    plt.plot(xss,gaus(xss,*param3),"-.",color="darkgreen",lw=3, label=l_cosmicsf)
    handles, labels = plt.gca().get_legend_handles_labels()
    order = [0,1,2,3,4,5] ## for just 2
    plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order]) 

    watermark = r'$\mathbf{ICARUS}\,$' + 'Data\nWork in Progress'
    plt.text(0.02, 0.92, watermark, fontsize=14, color='blue', alpha=1,
            ha='left', va='center', transform=plt.gca().transAxes)

    plt.xlabel("Time residuals [ns]",fontsize=12)
    plt.ylabel("Number of PMTs",fontsize=12)
    plt.title("Run {} with {} corrections".format(run,corr))

    plt.grid(linestyle="dashed",alpha=0.5)
    #name = "figs/run{}_base_vs_laseronly_vs_lasercosmics_w{}".format(run,corr)
    #plt.savefig(name+".pdf",dpi=200)
    plt.show()


### 8461: Compare old vs new cosmics corrections
Comparing both the residuals themselves, as well as their effect on validation samples.
if improvements across the board, replace in database with new values.
Need also to check dependance vs z,y...

Validation runs:
- 8552

In [ ]:
f1 = PATH + "Run_1/run8461_residuals_laseronly.csv"
f2 = COSMICSDB + "pmt_cosmics_timing_data_run08046_from8461.csv"

rmin=-10
rmax=10
r=(rmin,rmax)
s=0.3
b=int((rmax-rmin)/s)
plot_comparison(f1, f2, "New 8461 corr.", "Old 8461 corr.",b,r)
check_bias(f1, f2, "New 8461 corr.", "Old 8461 corr.")

In [ ]:
plot_validation(PATH,"Run_1",8552,8461)

In [ ]:
laser =  PATH + "Run_1/run8552_residuals_laseronly.csv"
cosmics = PATH + "Run_1/run8552_residuals_lasercosmics.csv"
check_bias(laser, cosmics, "laser corrected", "cosmics corrected")

## 9337

In [ ]:
f1 = PATH + "Run_2/run9337_residuals_laseronly.csv"
f2 = COSMICSDB + "pmt_cosmics_timing_data_run09301_from9337.csv"

rmin=-10
rmax=10
r=(rmin,rmax)
s=0.3
b=int((rmax-rmin)/s)
plot_comparison(f1, f2, "New 9337 corr.", "Old 9337 corr.",b,r)
check_bias(f1, f2, "New 9337 corr.", "Old 9337 corr.")

In [ ]:
plot_validation(PATH, "Run_2", 9441, 9337)
plot_validation(PATH, "Run_2", 9595, 9337)

In [ ]:
laser =  PATH + "Run_2/run9441_residuals_laseronly.csv"
cosmics = PATH + "Run_2/run9441_residuals_lasercosmics.csv"
check_bias(laser, cosmics, "laser corrected", "cosmics corrected")

## 9730

In [ ]:
f1 = PATH + "Run_2/run9730_residuals_laseronly.csv"
f2 = COSMICSDB + "pmt_cosmics_timing_data_run09628_from9730.csv"

rmin=-10
rmax=10
r=(rmin,rmax)
s=0.3
b=int((rmax-rmin)/s)
plot_comparison(f1, f2, "New 9730 corr.", "Old 9730 corr.",b,r)
check_bias(f1, f2, "New 9730 corr.", "Old 9730 corr.")

In [ ]:
plot_validation(PATH,"Run_2",9688,9730)
plot_validation(PATH,"Run_2",9764,9730)

## 10085

In [ ]:
f1 = PATH + "Run_2/run10085_residuals_laseronly.csv"
f2 = COSMICSDB + "pmt_cosmics_timing_data_run09773_from10085.csv"

rmin=-10
rmax=10
r=(rmin,rmax)
s=0.3
b=int((rmax-rmin)/s)
plot_comparison(f1, f2, "New 10085 corr.", "Old 10085 corr.",b,r)
check_bias(f1, f2, "New 10085 corr.", "Old 10085 corr.")

In [ ]:
plot_validation(PATH,"Run_2",9840,10085)
plot_validation(PATH,"Run_2",9974,10085)

In [ ]:
laser =  PATH + "Run_2/run9974_residuals_laseronly.csv"
cosmics = PATH + "Run_2/run9974_residuals_lasercosmics.csv"
check_bias(laser, cosmics, "laser corrected", "cosmics corrected")

## 12014

In [ ]:
plot_validation(PATH,"Run_3",11813,12014)
plot_validation(PATH,"Run_3",11816,12014)
plot_validation(PATH,"Run_3",11831,12014)
plot_validation(PATH,"Run_3",11843,12014)
plot_validation(PATH,"Run_3",11873,12014)
plot_validation(PATH,"Run_3",11933,12014)
plot_validation(PATH,"Run_3",11949,12014)

In [ ]:
laser =  PATH + "Run_3/run11949_residuals_laseronly.csv"
cosmics = PATH + "Run_3/run11949_residuals_lasercosmics.csv"
check_bias(laser, cosmics, "laser corrected", "cosmics corrected")

In [ ]:
laser =  PATH + "Run_3/run11813_residuals_laseronly.csv"
cosmics = PATH + "Run_3/run11813_residuals_lasercosmics.csv"
check_bias(laser, cosmics, "laser corrected", "cosmics corrected")